[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/monacofj/moeabench/blob/main/examples/example_07.ipynb)

# Example 07: Empirical Attainment Functions (EAF)

This example demonstrates how to use topo_attain surfaces to analyze 
the statistical distribution of Pareto fronts across multiple runs.
It visualizes the reliability bands of a search process.

In [ ]:
# Install MoeaBench from GitHub
!pip install --quiet git+https://github.com/monacofj/moeabench.git


In [ ]:



from MoeaBench import mb

print(f"MoeaBench v{mb.system.version()}")
print("--- Empirical Attainment Workshop ---\n")


### Setup: 2D problem for clear staircase visualization


In [ ]:
# 1. Setup: 2D problem for clear staircase visualization
mop1 = mb.mops.DTLZ2(M=2)
repeats = 10 

exp1 = mb.experiment()
exp1.name = "NSGA-II"
exp1.mop = mop1
exp1.moea = mb.moeas.NSGA2deap(population=100, generations=100)

print(f"Executing {exp1.name} ({repeats} runs)...")
exp1.run(repeat=repeats)


### Attainment Surfaces (Reliability Bands)


In [ ]:
# 2. Attainment Surfaces (Reliability Bands)
print("Calculating Attainment Surfaces (Optimistic, Median, Pessimistic)...")

# [mb.stats.topo_attainment] Based on Empirical Attainment Functions (EAF)
# surf contains:
#             .level           attainment level (0.5 = median)
#             .objectives      surface coordinates
#             .volume()        attained volume
surf1 = mb.stats.topo_attainment(exp1, level=0.1) # Best 10%
surf2 = mb.stats.topo_attainment(exp1, level=0.5) # Median
surf3 = mb.stats.topo_attainment(exp1, level=0.9) # Worst 10%
surf3.name = f"{exp1.name} (90% Worst)"

# Visualize the "Search Corridor" (Topographic Domain)
print("Plotting reliability band...")
mb.view.topo_shape(surf1, surf2, surf3, title="NSGA-II Search Corridor")


### Comparative Attainment (Topologic Gap)


In [ ]:
# 3. Comparative Attainment (Topologic Gap)
print(f"\nComparing with SPEA2...")
exp2 = mb.experiment()
exp2.name = "SPEA2"
exp2.mop = mop1
exp2.moea = mb.moeas.SPEA2(population=50, generations=100)
exp2.run(repeat=repeats)

# [mb.stats.topo_gap] Identifies spatial coverage differences
# res contains:
#             .surf1           attainment surface of exp1
#             .surf2           attainment surface of exp2
#             .volume_diff     difference in attained volumes
#             .report()        narrative summary
res1 = mb.stats.topo_gap(exp1, exp2, level=0.5)
print(res1.report())

# The diff object is iterable, returning (surf1, surf2) for plotting (Topographic Domain)
mb.view.topo_shape(*res1, title="Median Attainment: NSGA-II vs SPEA2")



### Interpretation

While Hypervolume gives a single number, Attainment Surfaces show *where* 
the algorithm succeeds or fails in objective space.

The reliability band (10%, 50%, 90%) reveals how much you can trust your 
results. A wide band means high variability; a narrow band means the 
algorithm is very consistent.

In the comparison, 'topo_gap' highlights the regions where one 
algorithm dominates the other's typical performance.